In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

import acquire
import prep
import split_scale

In [3]:
prep.get_base_df?

In [4]:
props = prep.get_base_df()

In [5]:
props.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18639 entries, 17295416 to 14214719
Data columns (total 4 columns):
bathroomcnt                     18639 non-null float64
bedroomcnt                      18639 non-null float64
calculatedfinishedsquarefeet    18639 non-null float64
taxvaluedollarcnt               18639 non-null float64
dtypes: float64(4)
memory usage: 728.1 KB


In [6]:
props.head(20)

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
parcelid,,,,
17295416,2.0,2.0,1144.0,239109.0
17208785,7.5,5.0,6278.0,1389230.0
17110420,1.5,3.0,1238.0,162481.0
17059646,1.5,3.0,1396.0,565000.0
17222974,5.5,3.0,4361.0,1562700.0
17294157,3.0,4.0,2846.0,269273.0
17290244,2.5,4.0,2542.0,1090401.0
17187503,2.0,3.0,2170.0,257517.0
17129832,3.0,6.0,2901.0,425115.0


In [8]:
props.describe()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
count,18639.000000,18639.000000,18639.000000,1.863900e+04
mean,2.329229,3.125168,1831.181126,5.115606e+05
std,0.970814,0.986794,952.169130,6.698696e+05
min,1.000000,1.000000,242.000000,1.050400e+04
25%,2.000000,2.000000,1214.000000,2.071050e+05
50%,2.000000,3.000000,1582.000000,3.685350e+05
75%,3.000000,4.000000,2179.000000,6.044115e+05
max,11.000000,11.000000,15450.000000,2.385837e+07
